In [ ]:
%load_ext autoreload
%autoreload 2

# Generate hard (near) negative name pairs

In [ ]:
from collections import namedtuple

import wandb

from src.data.utils import load_dataset
from src.models.triplet_loss import get_near_negatives

In [ ]:
given_surname = "surname"
Config = namedtuple("Config", "train_path near_negatives_path")
config = Config(
    train_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-train-unfiltered.csv.gz",
    near_negatives_path=f"s3://nama-data/data/processed/tree-hr-{given_surname}-near-negatives.csv.gz",
)

In [ ]:
wandb.init(
    project="nama",
    entity="nama",
    name="47_bad_pair_generator",
    group=given_surname,
    notes="",
    config=config._asdict()
)

In [ ]:
train = load_dataset(config.train_path)

input_names_train, weighted_actual_names_train, candidate_names_train = train

In [ ]:
k = 50
lower_threshold = 0.8  # 0.6
upper_threshold = 0.99 # 0.74
sample_size = 100 # 0
near_negatives_train = get_near_negatives(
    input_names_train, weighted_actual_names_train, candidate_names_train, 
    k=k, lower_threshold=lower_threshold, upper_threshold=upper_threshold,
    total=sample_size
)

In [ ]:
print("input_names_train", len(input_names_train))
# print("filtered_wans_train", sum(len(wan) for wan in filtered_wans_train))
print("candidate_names_train", len(candidate_names_train))
print("near_negatives_train", sum(len(negs) for _, negs in near_negatives_train.items()))

In [ ]:
for ix in range(0, 100):
    print(input_names_train[ix])
    wans = sorted(weighted_actual_names_train[ix], key=lambda x: -x[2])
    print("  positives", wans)
    print("  negatives", near_negatives_train[input_names_train[ix]])

In [ ]:
wandb.finish()